<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Введение" data-toc-modified-id="Введение-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Введение</a></span></li><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Применение-модели-faiss-для-тестовых-данных" data-toc-modified-id="Применение-модели-faiss-для-тестовых-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Применение модели faiss для тестовых данных</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Задача

**Задача:** разработать алгоритм, подбирающий для определённого товара набор из пяти похожих на него. Качество алгоритма оценить по метрике accuracy@5

**Предоставленные данные:**

* **base.csv** - анонимизированный набор товаров. Вектор признаков размерностью 72.

* **train.csv** - обучающий датасет. Каждая строчка - один товар, для которого известен уникальный id, вектор признаков и id товара из base.csv, который максимально похож на него (по мнению экспертов).

* **validation.csv** - датасет с товарами (уникальный id и вектор признаков), для которых надо найти наиболее близкие товары из base.csv

* **validation_answer.csv** - правильные ответы к предыдущему файлу.

# Тестирование модели

## Введение

На прошлом этапе мы построили оптимальную модель для поиска похожих товаров. Для обучения взяли библиотеку FAISS, а для масштабирования признаков использовали RobustScaler. Удалили мультиколлинеарные признаки, признаки с нессиметричным распределением и признаки, которые стояли на последнем месте по важности: 21, 25, 33, 70, 59, 65, 66. С помощью метода "локтя" и коэффициента силуэта определили оптимальное количество кластеров - 212. Далее подобрали количество посещаемых кластеров - 180.
На данном этапе применим построенный алгоритм на тестовых данных.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import faiss
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import RobustScaler
from matplotlib import colormaps
from sklearn.metrics import silhouette_score
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.metrics.pairwise import pairwise_distances_argmin
from sklearn.datasets import make_blobs
import category_encoders as ce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
pd.set_option('display.max_columns', 80)

In [4]:
k = 5

In [5]:
dict_base = {}
for i in range(72):
    dict_base[str(i)] = 'float32'
dict_base
dict_train = dict_base.copy()
dict_train['Target'] = 'str'

Загрузка датасетов.

In [6]:
df_base = pd.read_table("base.csv", index_col=0, sep=',', dtype=dict_base)
df_valid = pd.read_csv("validation.csv", index_col=0, dtype=dict_base)

In [7]:
df_valid.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100000-query,-57.372734,3.597752,-13.213642,-125.926788,110.745941,-81.279594,-461.003174,139.815720,112.880981,75.215752,-131.892807,-140.968567,-57.987164,-22.868887,150.895523,7.965574,17.622066,-34.868217,-216.138550,-80.908730,-52.579521,263.363129,56.266876,66.924713,21.609911,813.770081,-32.782940,20.794031,-79.779076,156.307083,-42.831329,-71.723335,83.283661,-304.174377,1.609402,55.834587,-29.474255,-139.162766,-126.038353,-62.643829,-5.012346,11.984921,-43.084946,190.123993,-24.996635,76.153900,-245.261566,-143.656479,-4.259628,-46.664196,-27.085403,-34.346962,75.530106,-47.171707,92.697319,60.475632,-127.486870,-39.484753,-124.384575,-307.949768,45.506813,-144.190948,-75.513023,52.830902,-143.439453,59.051933,69.282242,61.927513,111.592529,115.140656,-1099.130493,-117.079361
100001-query,-53.758705,12.790300,-43.268543,-134.417618,114.449913,-90.520126,-759.626038,63.995087,127.117905,53.128998,-153.717255,-63.951328,-52.369495,-33.390945,148.619507,-22.483829,15.164185,-56.202000,-153.614380,-79.831825,-101.055481,1203.537109,81.597130,101.018654,56.783424,92.209625,-126.860336,10.382887,-38.523361,165.383911,-77.840485,-169.538681,103.483238,-915.735718,16.109938,14.669937,-38.707085,-149.538376,-138.792923,-36.076176,-2.781422,2.283144,-142.477890,189.953949,-18.408230,90.517052,-95.530998,-259.636047,52.437836,-30.004599,14.502060,-1.071201,66.842667,-161.279892,94.794174,50.419983,-125.075256,-25.169033,-176.176880,-655.836914,-99.238373,-141.535217,-79.441833,29.185436,-168.605896,-82.872444,70.765602,-65.975952,97.077164,123.391640,-744.442322,-25.009319
100002-query,-64.175095,-3.980927,-7.679249,-170.160934,96.446159,-62.377739,-759.626038,87.477554,131.270111,168.920319,-220.309540,-31.378445,-8.788761,2.285323,133.266113,-41.309078,14.305538,-18.231812,-205.533707,-78.160309,-96.607674,1507.231323,-5.964200,34.937443,-56.086887,813.770081,-13.200474,18.966661,-35.110191,151.368500,-17.490252,-145.884293,15.533379,-655.395508,39.412827,62.554955,9.924992,-143.934616,-123.107796,-37.032475,-13.501337,12.913328,-116.038017,176.276154,-45.909943,103.491364,-90.656990,-162.615707,117.128235,13.079479,69.826889,-6.874451,63.707214,-123.851067,91.610817,59.760067,-129.566177,-12.822194,-154.197647,-407.199066,5.522629,-126.812973,-134.795410,37.368729,-159.662308,-119.232727,67.710442,86.002060,137.636414,141.081635,-294.052277,-70.969604
100003-query,-99.286858,16.123936,9.837166,-148.060440,83.697083,-133.729721,58.576405,-19.046659,115.042404,75.206734,-114.271957,-71.406456,-65.349319,24.377069,50.467300,-14.721335,15.069309,-46.682995,-176.604370,-78.690697,-139.227448,325.547119,3.632292,74.929504,-4.802103,813.770081,-52.982597,15.644382,-54.087467,151.309143,21.088570,-134.507889,65.118958,-529.295044,131.565521,67.642700,-22.884491,-145.906525,-86.917328,-11.863579,-22.188885,0.463720,-212.533752,170.522583,-48.092533,99.712555,-194.692413,-141.523178,60.217049,73.386383,118.567856,58.900810,55.569031,-181.091660,83.340485,66.083237,-114.048866,-57.156872,-56.335075,-318.680054,-15.984783,-128.101334,-77.236107,44.100494,-132.530121,-106.318985,70.883957,23.577892,133.183960,143.252945,-799.363647,-89.392670
100004-query,-79.532921,-0.364173,-16.027431,-170.884949,165.453918,-28.291668,33.931934,34.411217,128.903976,102.086914,-76.214172,-26.393860,34.423641,50.938889,157.683182,-23.786497,-33.175415,-0.592607,-193.318542,-79.651031,-91.889786,1358.481079,44.027733,121.527206,46.182999,433.623108,-82.233200,21.068508,-32.940117,149.268951,0.404718,-97.674530,81.719994,-825.644775,9.397169,49.359341,17.725466,-160.168152,-129.367950,-55.532898,-2.597821,-0.226103,-41.369141,92.090195,-58.626858,73.655441,-10.257370,-175.656784,

Также, для оценки качества нам нужно загрузить датасет с ответами.

In [8]:
df_answer = pd.read_csv("validation_answer.csv", index_col=0, dtype=dict_base)

In [9]:
df_answer.head(5)

,Expected
Id,
100000-query,2676668-base
100001-query,91606-base
100002-query,472256-base
100003-query,3168654-base
100004-query,75484-base


In [10]:
targets_valid = df_answer['Expected']

Удаление лишних признаков.

In [11]:
df_base = df_base.drop(['66', '25', '21', '33', '70', '59', '65'], axis=1)
df_valid = df_valid.drop(['66', '25', '21', '33', '70', '59', '65'], axis=1)

Масштабирование.

In [12]:
scaler = RobustScaler()

In [13]:
df_base_scaler = scaler.fit_transform(df_base)

In [14]:
df_valid_scaler = scaler.transform(df_valid)

## Применение модели faiss для тестовых данных

Размер индексных векторов будет равен количеству столбцов в датасете base. Количество кластеров - 212.

In [15]:
dims = df_base_scaler.shape[1] # размер индексных векторов
n_cells = 212 #количество кластеров
quantizer = faiss.IndexFlatL2(dims) # индекс для присвоения векторов определенному кластеру
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

Поиск будем вести по 180 кластерам.

In [16]:
idx_l2.nprobe = 180

In [17]:
idx_l2.train(np.ascontiguousarray(df_base_scaler[:500000, :]).astype('float32'))
idx_l2.add(np.ascontiguousarray(df_base_scaler).astype('float32'))

In [18]:
base_index = {l: v for l, v in enumerate(df_base.index.to_list())}

In [19]:
vecs, idx = idx_l2.search(np.ascontiguousarray(df_valid_scaler).astype('float32'), k)

In [20]:
acc = 0
for target, el in zip(targets_valid.values.tolist(), idx.tolist()):
    acc += int(target in [base_index[r] for r in el])

print(100 * acc / len(idx))

71.302


## Вывод

На данном этапе разработаный на основе FAISS алгоритм применили на тестовых данных. Результат работы алгоритма на тестовых данных - 71.302%. На тренировочных данных оценка метрики была 71.427%. Как видим, она почти не упала.